In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

In [2]:
# Set up the Selenium WebDriver to connect to the Selenium server (replace 'selenium' with your service name if different)
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode to avoid opening a UI
driver = webdriver.Remote(
    command_executor='http://selenium:4444/wd/hub',
    options=options
)

In [3]:
# Go to the Letterboxd popular movies page
driver.get("https://letterboxd.com/films/popular/page/1/")

In [5]:
# Find all elements that contain movie links
movie_elements = driver.find_elements(By.CSS_SELECTOR, 'a.frame')

# Extract URLs from the elements
movie_urls = [element.get_attribute('href') for element in movie_elements]

In [6]:
movie_urls

['https://letterboxd.com/film/barbie/',
 'https://letterboxd.com/film/parasite-2019/',
 'https://letterboxd.com/film/everything-everywhere-all-at-once/',
 'https://letterboxd.com/film/fight-club/',
 'https://letterboxd.com/film/la-la-land/',
 'https://letterboxd.com/film/interstellar/',
 'https://letterboxd.com/film/oppenheimer-2023/',
 'https://letterboxd.com/film/joker-2019/',
 'https://letterboxd.com/film/dune-2021/',
 'https://letterboxd.com/film/pulp-fiction/',
 'https://letterboxd.com/film/whiplash-2014/',
 'https://letterboxd.com/film/spider-man-into-the-spider-verse/',
 'https://letterboxd.com/film/get-out-2017/',
 'https://letterboxd.com/film/the-batman/',
 'https://letterboxd.com/film/the-dark-knight/',
 'https://letterboxd.com/film/saltburn/',
 'https://letterboxd.com/film/the-truman-show/',
 'https://letterboxd.com/film/spider-man-across-the-spider-verse/',
 'https://letterboxd.com/film/lady-bird/',
 'https://letterboxd.com/film/american-psycho/',
 'https://letterboxd.com/f

In [1]:
import argparse
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

def scrape_movie_urls(num_movies):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    movie_urls = []
    page = 1
    while len(movie_urls) < num_movies:
        driver.get(f"https://letterboxd.com/films/popular/page/{page}/")
        elements = driver.find_elements(By.CSS_SELECTOR, 'a.frame')
        for element in elements:
            if len(movie_urls) >= num_movies:
                break
            movie_urls.append(element.get_attribute('href'))
        page += 1
    
    df = pd.DataFrame(movie_urls, columns=['url'])
    return df
    #df.to_csv('/app/data/movie_urls.csv', index=False)


In [2]:
x = scrape_movie_urls(5)
x

,url
0,https://letterboxd.com/film/barbie/
1,https://letterboxd.com/film/parasite-2019/
2,https://letterboxd.com/film/everything-everywh...
3,https://letterboxd.com/film/fight-club/
4,https://letterboxd.com/film/la-la-land/


In [3]:
def normalize_rating(star_rating):
    rating_map = {'★': 0.2, '½': 0.1}
    return sum(rating_map[char] for char in star_rating if char in rating_map)

def scrape_reviews_per_movie(num_reviews_per_movie):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    print("Attempting to connect to Selenium WebDriver...")
    
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    print("Successfully connected to WebDriver!")
    
    movie_urls = pd.read_csv('/app/data/movie_urls.csv')['url'].tolist()
    all_reviews = []

    for movie_url in movie_urls:
        print(f"Scraping reviews for movie: {movie_url}")
        movie_reviews = []
        driver.get(f"{movie_url}/reviews/")
        
        page = 1
        while len(movie_reviews) < num_reviews_per_movie:
            review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.film-detail-content')
            
            for element in review_elements:
                if len(movie_reviews) >= num_reviews_per_movie:
                    break
                
                review_text_element = element.find_element(By.CSS_SELECTOR, 'div.body-text.-prose.collapsible-text')
                review_text = review_text_element.text if review_text_element else None
                
                rating_element = element.find_element(By.CSS_SELECTOR, 'span.rating')
                star_rating = rating_element.text if rating_element else None
                
                if review_text and star_rating is not None:
                    movie_reviews.append({
                        'movie_url': movie_url,
                        'review': review_text,
                        'rating': normalize_rating(star_rating)
                    })
            
            # Go to next page if needed
            if len(movie_reviews) < num_reviews_per_movie:
                page += 1
                driver.get(f"{movie_url}/reviews/page/{page}/")
            else:
                break
        
        all_reviews.extend(movie_reviews)
    
    df = pd.DataFrame(all_reviews)
    return df

    #df.to_csv('/app/data/movie_reviews.csv', index=False)

In [12]:
y = scrape_movie_urls(1)

In [13]:
y

,url
0,https://letterboxd.com/film/barbie/


In [ ]:
def normalize_rating(star_rating):
    rating_map = {'★': 0.2, '½': 0.1}
    return sum(rating_map[char] for char in star_rating if char in rating_map)

# Example usage:
normalized_reviews = [{'review': r['review'], 'rating': normalize_rating(r['rating'])} for r in reviews]
